### 🛒 Sales Sage

This project focuses on developing a machine learning model designed to predict the total sales for individual stores on a daily basis.

##### ⚠️ Important

Before continuing, ensure that you have generated the necessary data for analysis.
To do this, run the following command:

```bash
python3 get_data.py <year_from> <month_from> <day_from> <year_to> <month_to> <day_to>
```

---
### Importing libs
---

In [24]:
import sys

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

import datetime
import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

#sys.path.insert(0, '../')

#from src.proccess import DataProcessor
#from src.predict import ModelPredictor
#from src.train import ModelTrainer

---
### Variables
---

In [25]:
# Data Path
original_data_path = "../data/train-2023-08-01.csv"
process_data_path  = "../data/train-2023-08-01.parquet"
# predict_data_path  = "../data/bank_predict.csv"

# Model Path
model_path = "../models/model-2023-08-01.pkl"

In [26]:
df = pd.read_csv(original_data_path)
df.head()

,store_id,date,client_id,product_id,price
0,5000,2022-01-01,119016,1076,367.332895
1,5000,2022-01-01,130326,1512,361.560948
2,5000,2022-01-01,131705,1593,341.393517
3,5000,2022-01-01,191585,2456,360.698686
4,5000,2022-01-01,174376,1732,339.426975


---
### PreProccess Data
---

In [27]:
# Separate data in day, month and year columns 
# Create column sum(sales) on a day
# Create column weekday (0 = Segunda - 6 = Domingo)

In [28]:
new_df = df.groupby(['date', 'store_id'])['price'].sum().reset_index()
new_df = new_df.rename(columns={'price': 'total_sales'}) 
new_df.head(10)

,date,store_id,total_sales
0,2022-01-01,5000,28716.719986
1,2022-01-01,5001,8043.733717
2,2022-01-01,5002,11304.780167
3,2022-01-01,5003,40837.456326
4,2022-01-01,5004,57431.417801
5,2022-01-01,5005,40024.593189
6,2022-01-02,5000,47545.610893
7,2022-01-02,5001,3884.096941
8,2022-01-02,5002,10690.919754
9,2022-01-02,5003,43715.759864


In [29]:
# Create a weekday:
# 0 - Monday , 6 - Sunday
# 2022-01-01 -> Sunday - 6

def get_weekday(date: str):
    split_date = date.split('-')
    
    year = int(split_date[0])
    month = int(split_date[1])
    day = int(split_date[2])

    date = datetime.date(year, month, day)
    day_of_week = date.weekday()
    return day_of_week

new_df['weekday'] = new_df['date'].apply(get_weekday)
new_df.sample(5)  

,date,store_id,total_sales,weekday
1059,2022-06-26,5003,46329.103218,6
607,2022-04-12,5001,7978.780497,1
1095,2022-07-02,5003,47353.237113,5
429,2022-03-13,5003,39760.396457,6
1363,2022-08-16,5001,4005.862113,1


In [30]:
def get_day(date: str) -> int:
    split_date = date.split('-')
    day = split_date[-1]
    return int(day)
    
def get_month(date: str):
    split_date = date.split('-')
    month = split_date[1]
    return int(month)

def get_year(date: str):
    split_date = date.split('-')
    month = split_date[0]
    return int(month)

new_df['day'] = new_df['date'].apply(get_day)
new_df['month'] = new_df['date'].apply(get_month)
new_df['year'] = new_df['date'].apply(get_year)

In [31]:
new_df.sample(5)

,date,store_id,total_sales,weekday,day,month,year
1332,2022-08-11,5000,31800.182962,3,11,8,2022
1358,2022-08-15,5002,11253.327076,0,15,8,2022
122,2022-01-21,5002,12890.409667,4,21,1,2022
323,2022-02-23,5005,46227.281367,2,23,2,2022
3159,2023-06-11,5003,43433.625261,6,11,6,2023


In [32]:
# Drop columns:
new_df = new_df.drop('date', axis=1)
new_df.sample(5)

,store_id,total_sales,weekday,day,month,year
1633,5001,5553.614045,4,30,9,2022
2270,5002,8328.031494,5,14,1,2023
1800,5000,34762.254951,4,28,10,2022
1872,5000,37581.680936,2,9,11,2022
2127,5003,43704.108288,2,21,12,2022


In [34]:
# Adjust column order:
column_order = ['store_id', 'year', 'month', 'day', 'weekday', 'total_sales']
new_df = new_df.reindex(columns=column_order)
new_df.head()

,store_id,year,month,day,weekday,total_sales
0,5000,2022,1,1,5,28716.719986
1,5001,2022,1,1,5,8043.733717
2,5002,2022,1,1,5,11304.780167
3,5003,2022,1,1,5,40837.456326
4,5004,2022,1,1,5,57431.417801


In [35]:
# Save data as parquet:
new_df.to_parquet(process_data_path)

---
### Data Exploration
---

---
### Model
---

In [36]:
target_column_name = "total_sales"

X = new_df.drop(target_column_name, axis=1)
y = new_df[target_column_name]

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.3, random_state=1912
)

In [37]:
X.head()

,store_id,year,month,day,weekday
0,5000,2022,1,1,5
1,5001,2022,1,1,5
2,5002,2022,1,1,5
3,5003,2022,1,1,5
4,5004,2022,1,1,5


In [38]:
y.head()

0    28716.719986
1     8043.733717
2    11304.780167
3    40837.456326
4    57431.417801
Name: total_sales, dtype: float64

In [39]:
model = RandomForestRegressor(n_estimators=100, random_state=195)
model.fit(X_train, Y_train)

# Save the model
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

---
### Predict and Model Evaluation
---

In [40]:
# Read predict data:
prediction_path = "../data/predict-2023-08-03.parquet"
new_input_data = pd.read_parquet(prediction_path)
new_input_data.head()

,store_id,year,month,day,weekday
0,5000,2023,8,2,2
1,5000,2023,8,3,3
2,5001,2023,8,2,2
3,5001,2023,8,3,3
4,5002,2023,8,2,2


In [41]:
# read model and input data
model = pickle.load(open(model_path,'rb'))
new_input_data = pd.read_parquet(prediction_path)

In [43]:
# predict
print(" Make prediciton ...")
prediction = model.predict(new_input_data)
prediction

 Make prediciton ...


array([34283.28325542, 34172.52562456,  5155.01649127,  5610.35825963,
        9368.01329915,  9629.96611158, 44150.64068678, 81666.75925194,
       62021.16799031, 58889.25907313, 42336.93918718, 44476.37559196])

In [47]:
# Transform in dataframe:
prediction_df = pd.DataFrame({'Prediction': prediction})
prediction_df

,Prediction
0,34283.283255
1,34172.525625
2,5155.016491
3,5610.358260
4,9368.013299
5,9629.966112
6,44150.640687
7,81666.759252
8,62021.167990
9,58889.259073


In [48]:
# Save prediction
prediction_path_parque = "../data/predict-done-2023-08-03.parque"
prediction_df.to_parquet(prediction_path_parque)

## EXTRA FEATURES

In [37]:
# ------------------------------
# Get the feature importances
# ------------------------------
feature_importances = model.feature_importances_
print(feature_importances)

print(X.columns)

[0.77419555 0.12919034 0.0253869  0.06810725 0.00311996]
Index(['store_id', 'weekday', 'day', 'month', 'year'], dtype='object')


In [ ]:
def is_ohe_worth_it(df, column, threshold=10):
    """
    Check if One Hot Encoding is worth it for a categorical variable.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the variable
    - column (str): Name of the categorical variable
    - threshold (int, optional): Minimum number of unique values required for OHE (default=10)

    Returns:
    - bool: True if OHE is worth it, False otherwise
    """
    # Get the number of unique values in the column
    n_unique = df[column].nunique()

    # Check if the number of unique values exceeds the threshold
    if n_unique > threshold:
        print(f"Column '{column}' has {n_unique} unique values, which is above the threshold of {threshold}.")
        print("One Hot Encoding might be worth it.")
        return True
    else:
        print(f"Column '{column}' has {n_unique} unique values, which is below the threshold of {threshold}.")
        print("One Hot Encoding might not be worth it.")
        return False